# Kaggle Competition - TPS Feb 22

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

https://www.kaggle.com/c/tabular-playground-series-feb-2022/overview

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')
sns.set_palette('dark')

import matplotlib.pyplot as plt
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
data = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv',index_col='row_id')
data.head()

In [ ]:
data.target.value_counts()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
dft = data.groupby('target').agg('mean').T.plot(figsize=(20,10))


In [ ]:
dft = data.groupby('target').agg('skew').T.plot(figsize=(20,10))


In [ ]:
data

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
#from sklearn_extra.robust import RobustWeightedClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score


le = LabelEncoder()
data['target']= data[['target']].apply(le.fit_transform)

sc= RobustScaler()
data.iloc[:,:-1] = sc.fit_transform(data.iloc[:,:-1])

X= data.drop(columns='target',axis=1)
Y =data['target']

In [ ]:
rf = RandomForestClassifier(n_estimators=220, max_depth=None)
# svm = LinearSVC()
# lr = LogisticRegression(solver='newton-cg',max_iter=10000)
# gb = GradientBoostingClassifier(n_estimators=20, max_depth= 5,learning_rate=0.1)
#rwc = RobustWeightedClassifier(weighting="huber",loss="hinge", c=1.35,eta0=1e-3, max_iter=300)

model = rf
scores = []

i=1

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
sss.get_n_splits(X,Y)


def modelPerf(model,xtrain,ytrain,xtest,ytest):
    model.fit(xtrain,ytrain)
    ypred = model.predict(xtest)
    return accuracy_score(ytest,ypred)

for train_idx, test_idx in sss.split(X,Y):
    print(i)
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y.iloc[train_idx], Y.iloc[test_idx]
   
    
    scores.append(modelPerf(model,X_train,y_train,X_test,y_test))

    #break

    i+=1
    


In [ ]:
scores

### Test.csv

In [ ]:
data_test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv',index_col='row_id')
data_test.head()

In [ ]:
data_test1 = sc.transform(data_test)

y_pred_test = model.predict(data_test1)
y_out_test= le.inverse_transform(y_pred_test)


In [ ]:
ysubmit = pd.DataFrame(y_out_test,columns=['target'],index=data_test.index)
ysubmit.to_csv('submission.csv')